# 総合添削問題

教師あり学習（分類）の総合添削問題はこれまでの手法の特性とハイパーパラメーターのチューニングの重要性、そしてハイパーパラメーターのサーチ方法を理解しているかを問う問題となっています。  
使用するデータセットは手書きの数字の画像であり、判別しにくい数字もあることからパラメーターサーチや手法選択がより重要になります。

#### 問題

- 手書きの数字の認識・分類をするための学習器をより高い精度で作成したいと思います。
- 与えられるデータに対して手法を選び、ハイパーパラメーターを調整して学習能力の高い学習器を作ってください。
- また、一番評価の高い手法の名前と、調整したパラメーターとその値を出力してください。
----
- 採点基準は問題文の条件を満たしている状態で、以下の事項を考慮して総合的に採点いたします。
    - 評価値の高さ
    - パラメーターの調整方法
    - プログラムの実行時間
$- また、プログラムの実行時間は3分以内とします。

In [137]:
%%time

# 必要なモジュールがあれば追記してください
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

data = load_digits()
train_X, test_X, train_y, test_y = train_test_split(
    data.data, data.target, random_state=42)

# 以下にコードを記述してください

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

## LogisticRegression
model1 = LogisticRegression(random_state=42)
model1.fit(train_X, train_y)

## SVC
model2 = SVC(random_state=42)
model2.fit(train_X, train_y)

## LinearSVC
model3 = LinearSVC(random_state=42, max_iter=300000)
model3.fit(train_X, train_y)

## DecisionTreeClassifier
model4 = DecisionTreeClassifier()
model4.fit(train_X, train_y)

## RandomForestClassifier
model5 = RandomForestClassifier()
model5.fit(train_X, train_y)

## KNeighborsClassifier
model6 = KNeighborsClassifier(n_neighbors= 5)
model6.fit(train_X, train_y)


def display_the_best(models, test_X, test_y):
    scores = []
   
    for m in models:
        # get score
        score = m.score(test_X, test_y)
        
        # get f1 score
        pred_y = m.predict(test_X)
        true_y = test_y
#        print(pred_y)
        missing_datas = []
        for i in range(len(pred_y)):
            if pred_y[i] != true_y[i]:
                missing_datas.extend([{"index": i, "true_y": true_y[i], "pred_y": pred_y[i]}])
        f1 = f1_score(true_y, pred_y, average='macro') 
        print("%s: %s" % (m.__class__.__name__, f1))
        scores.extend([ { "model_name" : m.__class__.__name__, "model_parameters": m.get_params(deep=False), "score" : score, "f1_score": f1, "missing_datas": missing_datas} ])

    df = pd.DataFrame(scores, columns=["model_name", "model_parameters", "missing_datas", "score", "f1_score"]) 
    best_model = df.loc[df['f1_score'].idxmax()]
#    print(best_score["missing_datas"])
    print( "学習モデル: %s, パラメーター: %s, F1 :  %.4f"  %(best_model["model_name"], best_model["model_parameters"], best_model["f1_score"]))

# 最も成績のいいスコアを出力してください。
display_the_best([model1, model2, model3, model4, model5, model6, model7], test_X, test_y)    


LogisticRegression: 0.9667783570083582
SVC: 0.5937365791995064
LinearSVC: 0.9622544112644402
DecisionTreeClassifier: 0.850343420582349
RandomForestClassifier: 0.9569939429176506
KNeighborsClassifier: 0.9942462891080195
LogisticRegression: 0.9667783570083582
学習モデル: KNeighborsClassifier, パラメーター: {'algorithm': 'auto', 'leaf_size': 30, 'metric': 'minkowski', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}, F1 :  0.9942
CPU times: user 8.99 s, sys: 231 ms, total: 9.23 s
Wall time: 8.71 s


#### ヒント

- プログラムの実行時間を計測するにはプログラムの先頭に`%%time`と記述してください。
    - jupyter notebook上でのみ実行できるおまじないです。Pythonのプログラムでは実行できないので注意してください。
- モデルの手法の名前をプログラムで取得するには`model_name = model.__class__.__name__`とします。
- グリッドサーチ、ランダムサーチの結果得られるパラメーターセットを取得するには`best_params = clf.best_params_`とします。
- 最高評価を得たモデルが複数ある場合、そのうちの一つだけ出力すれば良いです。
- 採点時、モデルの評価にはF値という値を用います。
    - F値はprecision(精度)とrecall(再現率)という二つの評価値の調和平均です。  
    $$
    \begin{align}
    & \frac{2 ( precision \times recall )}{precision + recall}
    \end{align}
    $$
    - プログラムを作る際には`model.score(test_X, test_y)`で構いませんが、  
    自身で評価を確認したい場合はモデルに教師データを学習させたのち以下のようにします。
    ```python
    from sklearn.metrics import f1_score
    # モデルにデータを予測させる
    pred_y = clf.predict(testX)
    
    # モデルのF値を計算する
    score = f1_score(test_y, pred_y, average="micro")
    ```
- 今回のデータセットに関する情報は[UCI Machine Learning Repository(英語版サイト)](http://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits)を参照してください。
- 講座でいくつかのパラメーターを手法ごとに紹介しましたが、他にも調整可能なパラメーターが存在します。  
[scikit-learnのドキュメント(英語版サイト)](http://scikit-learn.org/stable/modules/classes.html)を参照してください。  
また、パラメーターに関してわからないことがあればチューターまでご質問ください。

####  解答例

添削課題の提出は以下のアドレスから提出いただきますようお願いします。<br>

https://goo.gl/forms/fW7CAspZMwHuWuqk2<br><br>
以下のアドレスからアンケートにご協力頂きたく存じます。<br>
ご回答のほど、よろしくお願いいたします。

https://goo.gl/forms/WHjJQYeodIndRvyz2